# Step 3, Separating Tasks and Resting State Data

In this step, we will separate the task-related data from the resting state data in our EEG recordings. This is crucial for focused analysis on specific cognitive tasks or resting state brain activity. Our focus is to segment the data based on event markers that indicate the start and end of tasks. These instructions are `instructed_toCloseEyes` and `instructed_toOpenEyes` for resting state data, and `task_start` and `task_end` for task-related data.

Here, we segment the data into two distinct sets, one for closed-eyes resting state and another for open-eyes resting state. This allows us to analyze the brain activity during these specific conditions separately.

In [ ]:
import mne
import os
import glob
import numpy as np

In [ ]:
# load processed fif file
processed_dir = os.path.join('..', '..', 'filtered_processed') # this can also be switch to 'processed' if 128 channels are preferred
resting_files = glob.glob(os.path.join(processed_dir, "*resting_state*.fif"))

print(f"Found {len(resting_files)} resting state files:")
for f in resting_files:
    print(f"  - {os.path.basename(f)}")

# load all files
raw_list = []
for file_path in resting_files:
    print(f"Loading {os.path.basename(file_path)}...")
    r = mne.io.read_raw_fif(file_path, preload=True)
    raw_list.append(r)

In [ ]:
# extract events for each raw file separately using event names
task_1_events_list = []  # Eyes CLOSED
task_2_events_list = []  # Eyes OPEN

for raw in raw_list:
    events, event_dict = mne.events_from_annotations(raw)
    
    close_eyes_id = event_dict.get('instructed_toCloseEyes')
    open_eyes_id = event_dict.get('instructed_toOpenEyes')
    
    # extract eyes CLOSED events using event name
    task_1 = events[events[:, 2] == close_eyes_id]
    task_1_events_list.append(task_1)
    
    # extract eyes OPEN events using event name
    task_2 = events[events[:, 2] == open_eyes_id]
    task_2_events_list.append(task_2)

print(f"  Eyes Closed (instructed_toCloseEyes): {sum(len(e) for e in task_1_events_list)} events")
print(f"  Eyes Open (instructed_toOpenEyes): {sum(len(e) for e in task_2_events_list)} events")

In [ ]:
# method to extract segments from multiple raw files

def extract_and_save_segments_from_multiple(raw_list, events_list, event_name, segment_duration=1.0, output_dir='../../gan_data'):
    all_segments = []
    
    for file_idx, (raw, events) in enumerate(zip(raw_list, events_list)):
        sfreq = raw.info['sfreq']
        segment_samples = int(sfreq * segment_duration)
        
        print(f"\nFile {file_idx + 1}/{len(raw_list)}: {len(events)} events")
        
        segments = []
        
        for event in events:
            start_sample = event[0]
            end_sample = start_sample + segment_samples
            
            if end_sample <= raw.n_times:
                segment = raw.get_data(start=start_sample, stop=end_sample)
                segments.append(segment)
        
        all_segments.extend(segments)
    
    # combine segments
    all_segments = np.array(all_segments)
    
    # normalize segments
    normalized_segments = []
    
    for segment in all_segments:
        
        mean = segment.mean()
        std = segment.std() + 1e-8
        normalized = (segment - mean) / std
    
        normalized_segments.append(normalized)
    
    normalized_segments = np.array(normalized_segments).astype(np.float32)
    
    os.makedirs(output_dir, exist_ok=True)
    
    n_segments, n_channels, n_timepoints = normalized_segments.shape
    concatenated_data = normalized_segments.transpose(1, 0, 2).reshape(n_channels, n_segments * n_timepoints)
    info = mne.create_info(ch_names=raw_list[0].ch_names, sfreq=raw_list[0].info['sfreq'], ch_types='eeg')
    raw_segments = mne.io.RawArray(concatenated_data, info)
    
    data_path_fif = os.path.join(output_dir, f'{event_name}_segments.fif')
    raw_segments.save(data_path_fif, overwrite=True)
    print(f"Saved FIF: {data_path_fif}")
    
    individual_dir = os.path.join(output_dir, event_name + '_individual')
    os.makedirs(individual_dir, exist_ok=True)
    
    print(f"Saving individual segments to {individual_dir}...")
    for idx, segment in enumerate(normalized_segments):
        segment_path = os.path.join(individual_dir, f'{event_name}_segment_{idx:04d}.npy')
        np.save(segment_path, segment)
    
    print(f"Saved {len(normalized_segments)} individual segment files")
    print(f"Format: {event_name}_segment_0000.npy, {event_name}_segment_0001.npy, ...")
    
    # save metadata
    metadata = {
        'n_segments': len(normalized_segments),
        'n_channels': normalized_segments.shape[1],
        'n_timepoints': normalized_segments.shape[2],
        'sfreq': raw_list[0].info['sfreq'],
        'segment_duration': segment_duration,
        'ch_names': raw_list[0].ch_names,
        'event_name': event_name,
        'n_files': len(raw_list),
        'individual_segments_dir': individual_dir
    }
    
    metadata_path = os.path.join(output_dir, f'{event_name}_metadata.npy')
    np.save(metadata_path, metadata)
    print(f"Saved metadata: {metadata_path}")
    
    return normalized_segments, metadata

# extract and save EYES CLOSED segments from all files
eyes_closed_segments, eyes_closed_meta = extract_and_save_segments_from_multiple(
    raw_list=raw_list,
    events_list=task_1_events_list,
    event_name='eyes_closed',
    segment_duration=1.0
)

# extract and save EYES OPEN segments from all files
eyes_open_segments, eyes_open_meta = extract_and_save_segments_from_multiple(
    raw_list=raw_list,
    events_list=task_2_events_list,
    event_name='eyes_open',
    segment_duration=1.0
)